In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection # New import
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from STK_Sim import *
%matplotlib tk

In [11]:
Filename = 'AERO_401_Initial_Assessment'
stk_object = STK_Simulation(False,Filename)

In [108]:
# Setting scenario variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
dt = datetime.timedelta(days=5, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+dt).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")
dt = 60 # Computation interval
n_targets = 15
n_sats = 1

In [14]:
# Generating Targets

# Generating a polygon to bound the lat/lon coordinates, you can create your polygon, in the same format as Targets_Polygon.txt.
poly = Create_Poly('Input_Files/Targets_Polygon.txt')

# Writing random points within the polygon to a target file.
targets_filename = 'Input_Files/Targets_File.txt'
polygon_random_points(poly,n_targets).to_csv(targets_filename,index=False)

# Loading targets into stk from file.
targets_filename = 'Input_Files/Targets_File.txt'
stk_object.Target_Loader(targets_filename)

In [158]:
lower = [500,0,0,0,0]
upper = [615,180,180,180,180]

X = []
y = []
for i in range(100):
    Individual = np.random.uniform(low=lower, high=upper, size=(n_sats,5))
    X.append(Individual)
    file = open("Input_Files/Satellites_File.txt","w")
    file.write("Per,Apo,Inc,AoP,Asc,Loc,Tar,Aft\n")
    nvars = 5
    for n in range(len(Individual)):
        Alt = Individual[n,0]
        Inc = Individual[n,1]
        AoP = Individual[n,2]
        Asc = Individual[n,3]
        Loc = Individual[n,4]
        Tar = 1
        file.write(f"{Alt},{Alt},{Inc},{AoP},{Asc},{Loc},{Tar}\n")
    file.close()
    satellites_filename = 'Input_Files/Satellites_File.txt'
    stk_object.Satellite_Loader(satellites_filename)
    stk_object.Compute_AzEl(dt)
    num_total_angles = 0
    for t in range(len(stk_object.targets)):
        num_total_angles += len(np.where(stk_object.Azimuth_vs_Elevation[f"Target{t+1}"]>0)[0])
    y.append(100*num_total_angles/324/len(stk_object.targets))
X = np.array(X)
X = np.reshape(X,[-1,5])

[========================================] 15/15 [100%] in 0.3s (48.82/s)       
[========================================] 15/15 [100%] in 0.6s (25.10/s)       
[========================================] 15/15 [100%] in 0.2s (113.90/s)      
[========================================] 15/15 [100%] in 0.6s (26.43/s)       
[========================================] 15/15 [100%] in 0.5s (29.31/s)       
[========================================] 15/15 [100%] in 0.2s (105.67/s)      
[========================================] 15/15 [100%] in 0.5s (31.03/s)       
[========================================] 15/15 [100%] in 0.2s (52.54/s)       
[========================================] 15/15 [100%] in 0.5s (31.04/s)       
[========================================] 15/15 [100%] in 0.5s (29.22/s)       
[========================================] 15/15 [100%] in 0.5s (29.56/s)       
[========================================] 15/15 [100%] in 0.6s (28.58/s)       
[===========================

In [182]:
kernel = RBF(length_scale=.001*np.array([1,1,1,1,1]))
gpr = GaussianProcessRegressor(kernel=kernel)
# Fit the model and make predictions
gpr.fit(X,y)
X_pred = np.random.uniform(low=lower, high=upper, size=(10,5))
y_pred,y_std = gpr.predict(X_pred,return_std=True)
for i in range(10):
    print(f"i:{i}|x:{X_pred[i]}|y:{y_pred[i]}|std:{y_std[i]}")

i:0|x:[548.23038213  19.42021024   1.5919953   62.38350278  71.75146419]|y:0.0|std:1.0
i:1|x:[592.52901343  79.09107349  81.63570475 145.70780926 173.63732519]|y:0.0|std:1.0
i:2|x:[571.58762585   1.67083926 171.35623475  74.77556757  18.07610029]|y:0.0|std:1.0
i:3|x:[552.9489647  154.48001871  51.66815607 176.40863299 137.0838959 ]|y:0.0|std:1.0
i:4|x:[526.48235149 173.276448    25.63003573  60.02617904  91.68123538]|y:0.0|std:1.0
i:5|x:[523.36342105 111.95975549  88.5682034   52.6019454  131.88860983]|y:0.0|std:1.0
i:6|x:[580.68404349 167.54401738  21.04091433  57.5406649  127.52292824]|y:0.0|std:1.0
i:7|x:[605.33111269 137.31819264 144.93166406 126.04161839 168.17132199]|y:0.0|std:1.0
i:8|x:[514.56507504  80.16484197 160.86642956 129.95498812 127.37508561]|y:0.0|std:1.0
i:9|x:[508.14031455 116.30182444   5.81555073 152.83583574  32.55192444]|y:0.0|std:1.0
